In [115]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 1000000 

%matplotlib inline
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'NanumBarunGothic'
plt.rcParams["font.size"] = 20
plt.rcParams["figure.figsize"] = (15,8)

mpl.rc('axes', labelsize=12)
mpl.rc('xtick', labelsize=8)
mpl.rc('ytick', labelsize=8)

import seaborn as sns
#corr_set
from itertools import product


In [72]:
#check eup-dong list
sample = pd.read_csv('data/2.세종시_표제부.csv')
sample['읍면동'] = sample['대지위치'].str.replace('세종특별자치시 ','')
sample['읍면동'] = sample['읍면동'].str.replace('조치원읍 ','조치원읍')
sample[['1', '2','3']] = sample['읍면동'].str.split(' ', n=2, expand=True)
sample['2'] = sample['2'].str.replace('조치원읍','조치원읍 ')

#set myn&dong
list_myn = ['소정면', '연서면', '전의면', '금남면','전동면','장군면','연동면','연기면','부강면']
list_dong = ['종촌동', '소담동', '보람동','다정동', '도담동', '합강동', '가람동','반곡동','나성동', 
              '한솔동','산울동','집현동','어진동','고운동', '아름동', '해밀동', '대평동','새롬동']

In [73]:
def make_rentfee_df(a,b,c) :
  df = pd.read_csv(a)
  df['보증금(만원)'] = df['보증금(만원)'].str.replace(',','')
  df = df.astype({'보증금(만원)' : float, '계약년월' : 'str'})
  df['시군구'] = df['시군구'].str.replace('세종특별자치시 ','')
  df.replace({'전세' : 'yr', '월세' : 'mth'},inplace=True)
  
  df_j = make_rentfee_data(df,b,'yr')
  df_m = make_rentfee_data(df,b,'mth')
  
  d = pd.merge(c,df_j, how='outer',on='읍면동')
  f = pd.merge(d,df_m, how='outer',on='읍면동')

  return f

def make_rentfee_data(i, b, c) :

  df = i[i['전월세구분'] == c]
  df[['읍면동', '2']] = df['시군구'].str.split(' ', n=1, expand=True)

  df = df.groupby(['읍면동']).agg({'계약일' : 'count', '보증금(만원)':'mean'})
  x = b+'_'+c+'_cnt'
  y = b+'_'+c+'_price'
  
  df.columns = [x,y]

  return df

In [85]:
ofi = make_rentfee_df('data/10.세종시_오피스텔(전월세)_실거래가.csv','ofi',data_d)
apt = make_rentfee_df('data/7.세종시_아파트(전월세)_실거래가.csv','apt',data_d)
ali = make_rentfee_df('data/8.세종시_연립다세대(전월세)_실거래가.csv','ali',data_d)
# 단독주택 제외 >> 대지단위(명확하게 비교하기가 어려움)
summary = pd.merge(ofi, apt, how='outer', on='읍면동')
summary = pd.merge(summary, ali, how='outer', on='읍면동')

In [96]:
# 상권코드 정리
market = pd.read_csv('data/14.세종시_상권정보.csv')
code = pd.read_csv('data/15.세종시_상권정보_업종코드.csv')
list_main = list(set(code['대분류코드']))
list_middle = list(set(code['중분류코드']))
list_middle = sorted(list_middle)

In [105]:
def make_summary_by_location(a) :
    list_data = list_dong if a == 'dong' else list_myn
    a = summary[summary['읍면동'].isin(list_data)]
    a = a.reset_index(drop=True)

    ex = pd.DataFrame()

    #market divieded
    for i in a['읍면동'] :
        df = market[market['행정동명']==i]
        df = df.groupby('상권업종중분류코드').agg({'행정동명':'count','상권업종중분류명' : 'first','상권업종대분류코드' :'first','상권업종대분류명' : 'first'})
        
        df_T= df.transpose()
        df_T['읍면동'] = i
        df_T['total'] = df['행정동명'].sum()

        list_x = df_T.iloc[0]

        ex = ex.append(pd.Series(list_x), ignore_index=True)
        
    x = pd.merge(a,ex, how='outer', on='읍면동')
    
    return x

In [109]:
# Set the data without nan at market data
dong = make_summary_by_location('dong')
dong_final = dong[dong['total'] != 0]
myn = make_summary_by_location('myn')
myn_final = myn[myn['total'] != 0]

# drop ofi
dong_final = dong_final.sort_index(axis=1)
dong_final = dong_final.drop({'ofi_mth_price','ofi_mth_cnt','ofi_yr_price','ofi_yr_cnt'},axis = 1)

myn_final = myn_final.sort_index(axis=1)
myn_final = myn_final.drop({'ofi_mth_price','ofi_mth_cnt','ofi_yr_price','ofi_yr_cnt'},axis = 1)

In [66]:
check = code[code['대분류코드']=='N']
check['code'] = check['중분류코드'] + check['중분류명']
list_check = list(set(check['code']))
list_check = sorted(list_check)
list_check

<ipython-input-66-4b4dd3542c68>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['code'] = check['중분류코드'] + check['중분류명']


['N01PC/오락/당구/볼링등',
 'N02무도/유흥/가무',
 'N03연극/영화/극장',
 'N04스포츠/운동',
 'N05요가/단전/마사지',
 'N06전시/관람',
 'N07놀이/여가/취미',
 'N08경마/경륜/성인오락']

In [110]:
for i in list_main:
    df_dong01 = dong_final.loc[:,'ali_mth_cnt':]
    df_myn01 = dong_final.loc[:,'ali_mth_cnt':]
    
    list_x = [middle for middle in list_middle if i in middle]
    print(list_x[0],list_x[-1])

    df_dong02 = dong_final.loc[ :,list_x[0]:list_x[-1]]
    df_dong02['읍면동'] = dong_final['읍면동']
    
    df_myn02 = dong_final.loc[ :,list_x[0]:list_x[-1]]
    df_myn02['읍면동'] = dong_final['읍면동']

    corr01 = pd.merge(df01,df02,on='읍면동')
    corr02 = pd.merge(df01,df02,on='읍면동')
    print(i, list(set(code[code['대분류코드']==i]['대분류명'])))


    corr_dong = corr01.corr(method = 'pearson')
    corr_myn = corr02.corr(method = 'pearson')
    corr_dong.to_csv('png/{}'.format(i)+'_dong.csv')
    corr_myn.to_csv('png/{}'.format(i)+'_myn.csv')
    #df_heatmap = sns.heatmap(corr)
    #plt.savefig('png/{}'.format(i)+'.png')

    #plt.clf()


O01 O05
O ['숙박']
D01 D26
D ['소매']
R01 R20
R ['학문/교육']
Q01 Q15
Q ['음식']
L01 L05
L ['부동산']
P01 P03
P ['스포츠']
N01 N08
N ['관광/여가/오락']
F01 F20
F ['생활서비스']


In [112]:
def print_corr_set(i) :
    df = pd.read_csv('png/{}'.format(i)+'_dong.csv')

In [114]:
df = pd.read_csv('png/D_Dong.csv')
df

,Unnamed: 0,ali_mth_cnt,ali_mth_price,ali_yr_cnt,ali_yr_price,apt_mth_cnt,apt_mth_price,apt_yr_cnt,apt_yr_price,total,...,F07,F08,F09,F11,F12,F13,F14,F15,F16,F20
0,ali_mth_cnt,1.000000,-0.998497,-1.0,1.0,0.903559,0.856464,0.830447,0.995610,0.787496,...,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN
1,ali_mth_price,-0.998497,1.000000,-1.0,1.0,-0.878716,-0.883471,-0.798663,-0.999244,-0.752531,...,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN
2,ali_yr_cnt,-1.000000,-1.000000,1.0,-1.0,-1.000000,1.000000,-1.000000,1.000000,-1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ali_yr_price,1.000000,1.000000,-1.0,1.0,1.000000,-1.000000,1.000000,-1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,apt_mth_cnt,0.903559,-0.878716,-1.0,1.0,1.000000,0.011590,0.763902,0.392595,0.372659,...,-0.189506,NaN,NaN,0.346951,0.118467,-0.298323,-0.073163,NaN,-0.999943,NaN
5,apt_mth_price,0.856464,-0.883471,1.0,-1.0,0.011590,1.000000,-0.351454,0.390511,0.081457,...,-0.383735,NaN,NaN,0.510238,-0.102126,-0.904461,-0.857685,NaN,0.633893,NaN
6,apt_yr_cnt,0.830447,-0.798663,-1.0,1.0,0.763902,-0.351454,1.000000,0.197144,0.306649,...,-0.352592,NaN,NaN,-0.033548,0.278406,-0.098446,0.554435,NaN,-0.853837,NaN
7,apt_yr_price,0.995610,-0.999244,1.0,-1.0,0.392595,0.390511,0.197144,1.000000,0.614235,...,0.892213,NaN,NaN,0.338988,0.544152,0.682399,0.146254,NaN,-0.117141,NaN
8,total,0.787496,-0.752531,-1.0,1.0,0.372659,0.081457,0.306649,0.614235,1.000000,...,0.349555,NaN,NaN,0.812399,-0.298061,0.047783,0.091688,NaN,-0.685925,NaN
9,F01,0.968372,-0.953240,-1.0,1.0,0.302848,0.074456,0.252707,0.457940,0.893904,...,0.424094,NaN,NaN,0.955208,-0.353798,-0.172152,-0.136646,NaN,-0.727365,NaN
